In [1]:
import sys
import warnings
sys.path.append("../")
import pandas as pd
from pandas import DataFrame
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
import datetime
import importlib
from typing import Tuple
import arbitrage_system as ar

### Load symbol for trading

In [12]:
%store -r tickers

In [3]:
symbols = []
for ticker in tickers:
    symbol = ticker["symbol"]
    if str(symbol).endswith("USDT"):
        symbols.append(symbol)

In [4]:
len(symbols)

145

In [5]:
TRADE_TIMEFRAME = "4h"
TRADE_TIMEFRAME_IN_MS = 4 * 60 * 60 * 1000
GRANULAR_TIMEFRAME = "5m"
HOLDING_PERIOD = 14
BASE_PAIR = "BTCUSDT"

#### Loading trading data

In [9]:
importlib.reload(ar)

<module 'arbitrage_system' from '/home/andy/CryptoTradingPlatform/TraderRobot/project/arbitrage/arbitrage_system.py'>

In [8]:
dfs = ar.import_trading_data(TRADE_TIMEFRAME,tickers)
gdfs = ar.import_granular_data(GRANULAR_TIMEFRAME,tickers)

...loading trading data...
CFXUSDT SSVUSDT SOLUSDT ATAUSDT UNFIUSDT KAVAUSDT APTUSDT DUSKUSDT EOSUSDT 1000XECUSDT 1000SHIBUSDT CHRUSDT YFIUSDT SNXUSDT FETUSDT THETAUSDT LINAUSDT DENTUSDT SXPUSDT LINKUSDT SPELLUSDT API3USDT ZRXUSDT OMGUSDT ZILUSDT ROSEUSDT BALUSDT XMRUSDT SKLUSDT MINAUSDT FILUSDT XTZUSDT AAVEUSDT OPUSDT GMXUSDT ASTRUSDT LDOUSDT ALICEUSDT ZENUSDT MTLUSDT JASMYUSDT GALAUSDT LUNA2USDT LTCUSDT MANAUSDT INJUSDT MASKUSDT APEUSDT CELRUSDT RENUSDT ADAUSDT FTMUSDT QNTUSDT MATICUSDT ANTUSDT COTIUSDT TUSDT CVXUSDT WAVESUSDT FLMUSDT FXSUSDT BNBUSDT NEARUSDT 1INCHUSDT AUDIOUSDT GRTUSDT RUNEUSDT LITUSDT EGLDUSDT ONEUSDT ARPAUSDT IMXUSDT XRPUSDT RNDRUSDT ONTUSDT REEFUSDT ANKRUSDT GMTUSDT ACHUSDT HOTUSDT DYDXUSDT PEOPLEUSDT KSMUSDT LRCUSDT MAGICUSDT ZECUSDT IOSTUSDT BTCDOMUSDT GALUSDT ETCUSDT QTUMUSDT DOTUSDT PHBUSDT TRXUSDT KNCUSDT CKBUSDT BLUEBIRDUSDT ENSUSDT UNIUSDT CRVUSDT IOTXUSDT RLCUSDT FOOTBALLUSDT BTCUSDT DARUSDT VETUSDT ARUSDT DEFIUSDT ALGOUSDT TOMOUSDT WOOUSDT HOOKUSDT FLOWU

##### Get key info of the trading data such as length, last index, completeness

In [10]:
stat_df = ar.examine_data(TRADE_TIMEFRAME,dfs)

##### Prepare data

In [ ]:
def plot_results(trade_df:DataFrame, market_trend:DataFrame,timerange:Tuple[int,int]):
    fig,ax = plt.subplots()
    
    fig.set_dpi(720)
    fig.set_size_inches(10,6)
    
    for i in market_trend.index:
        row = market_trend.loc[i,:]
        start = i
        end = row["end"]
        color = "tab:green" if row["value"] > 0 else "tab:red"
        ax.axvspan(start, end, color=color, alpha=0.1,linewidth=1)  #type: ignore
    
    draw_df = trade_df[(trade_df["open"]>=timerange[0])&(trade_df["open"]<=timerange[1])].copy()
    
    ax.plot(draw_df["open"],draw_df["cumsum_profit"],c="b",linewidth=1,label="cumsum_profit")
    ax.plot(draw_df["open"],draw_df["cumsum_long_profit"],c="g",linewidth=1,label="cumsum_long_profit")
    ax.plot(draw_df["open"],draw_df["cumsum_short_profit"],c="r",linewidth=1,label="cumsum_short_profit")
    ax.plot(draw_df["open"],draw_df["cumsum_trading_cost"],c="y",linewidth=1,label="cumsum_trading_cost")
    
    highest = max(draw_df["cumsum_long_profit"].max(),draw_df["cumsum_long_profit"].max(),draw_df["cumsum_profit"].max())
    btc_highest = draw_df["BITCOIN_PRICE"].max()
    
    ax.plot(draw_df["open"],draw_df["BITCOIN_PRICE"]*highest/btc_highest,color="tab:gray",linewidth=1,label="BITCOIN")
    ax.plot(draw_df["open"],[0 for i in range(len(draw_df))],color="black",linewidth=1)
    
    plt.show()

In [ ]:
def backtest_strategy(
    trade_duration:int,
    leverage:float,
    pair_number:int,
    holding_period:int,
    verbose:int,
    balance:int,
    trading_fee:float,
    take_profit_long: float,
    stop_loss_long:float,
    take_profit_short: float,
    stop_loss_short:float,
    altcoin_cycle:str,
    market_cycle:str,
    plot:bool = True,
    ):


    selected_pairs = stat_df[(stat_df["length"]>trade_duration+holding_period)&\
        (stat_df["coverage"]==1)]["symbol"].values.tolist()
    
    print("...preprocessing, calculate MACD ...")
    merged_df, pairs = ar.calculate_macd(
        pairs=selected_pairs,
        dfs=dfs
    )

    print("...preprocessing, calculate MACD diff with base  ...")
    merged_df = ar.calculate_macd_diff(
        dataframe=merged_df,
        pairs=pairs
    )

    trade_end = int(merged_df.index.max() - holding_period * TRADE_TIMEFRAME_IN_MS)
    trade_start = int(min(max(trade_end - trade_duration * TRADE_TIMEFRAME_IN_MS, merged_df.index.min()),trade_end))

    #Check if any missing open-time index
    for i in range (trade_start,trade_end,TRADE_TIMEFRAME_IN_MS):
        if not i in merged_df.index:
            print(f"miss {i}")

    btc_init_price = merged_df.loc[trade_start,:]["BITCOIN_PRICE"]

    trade_df = pd.DataFrame({"open":[trade_start],"datetime":[""], "symbol":[""],"type":[""], "amount":[0], "profit":[0],\
        "cost":[0],"balance":[balance], "close_type":[""], "BITCOIN_PRICE":[btc_init_price]})

    balance = balance
    profit_before_cost = 0

    print("...backtesting...")
    open_unix_time = trade_start

    while open_unix_time < trade_end:
        
        date_time = datetime.datetime.utcfromtimestamp(open_unix_time/1000).strftime('%Y-%m-%d %H:%M:%S')
        row = merged_df.loc[open_unix_time,:].copy()
        
        #Executing trade
        trade_pairs = []
        trade_types = []
        trade_costs = []
        trade_profits = []
        invest_amounts = []
        
        balances = []
        BITCOIN_PRICEs = []
        close_types = []

        long_pairs, short_pairs, long_dict, short_dict =\
            ar.arbitrate_both_sides(
            row=row,
            market_cycle=market_cycle,
            altcoin_cycle=altcoin_cycle,
            n_pairs=pair_number,
            pairs=pairs)

        short_budget = long_budget = leverage * balance/2
        trade_num = len(long_pairs) + len(short_pairs)
        enter_time = open_unix_time + TRADE_TIMEFRAME_IN_MS
        end_time = open_unix_time + (1+holding_period) * TRADE_TIMEFRAME_IN_MS
        
        invest_amount = long_budget/len(long_pairs)
        for i in range(len(long_pairs)):
            pair = long_pairs[i]
            status = ar.check_stoploss(
                pair = pair,
                enter_time = enter_time,
                end_time = end_time,
                direction = "LONG",
                take_profit=take_profit_long,
                stop_loss=stop_loss_long,
                trading_df_dict=dfs,
                granular_df_dict=gdfs
            )
            
            match status:
                case 0:
                    cumsum_log_return = merged_df.loc[enter_time:end_time,:][f"{pair}_LOG_RETURN"].sum()
                case -1:
                    cumsum_log_return = np.log(1-stop_loss_long)
                case 1:
                    cumsum_log_return = np.log(1+take_profit_long)
            
            profit_before_cost = invest_amount * (np.exp(cumsum_log_return)-1)
            cost = invest_amount * 2 * trading_fee
            profit = profit_before_cost - cost
            balance += profit #type: ignore
            trade_pairs.append(pair)
            trade_types.append("long")
            trade_costs.append(cost)
            trade_profits.append(profit)
            invest_amounts.append(invest_amount)
            balances.append(balance)
            close_types.append(status)
            BITCOIN_PRICEs.append(row["BITCOIN_PRICE"])
        
        invest_amount = short_budget/len(short_pairs) if len(short_pairs)>0 else 0
        for i in range(len(short_pairs)):
            pair = short_pairs[i]
            status = ar.check_stoploss(
                pair = pair,
                enter_time = enter_time,
                end_time = end_time,
                direction = "SHORT",
                take_profit=take_profit_short,
                stop_loss=stop_loss_short,
                trading_df_dict = dfs,
                granular_df_dict = gdfs
            )
            
            match status:
                case 0:
                    cumsum_log_return = -merged_df.loc[enter_time:end_time,:][f"{pair}_LOG_RETURN"].sum()
                case -1:
                    cumsum_log_return = np.log(1-stop_loss_short)
                case 1:
                    cumsum_log_return = np.log(1+take_profit_short)
            
            profit_before_cost = invest_amount * (np.exp(cumsum_log_return)-1)
            cost = invest_amount * 2 * trading_fee
            profit = profit_before_cost - cost
            balance += profit #type: ignore
            trade_pairs.append(pair)
            trade_types.append("short")
            trade_profits.append(profit)
            invest_amounts.append(invest_amount)
            trade_costs.append(cost)
            balances.append(balance)
            close_types.append(status)
            BITCOIN_PRICEs.append(row["BITCOIN_PRICE"])

        times = [open_unix_time for i in range(trade_num)]
        date_times = [date_time for i in range(trade_num)]

        new_rows =pd.DataFrame({"open": times, "datetime": date_times, "symbol":trade_pairs,\
            "type":trade_types,"amount":invest_amounts, "profit":trade_profits, \
                "cost":trade_costs,"balance":balances, "close_type":close_types, "BITCOIN_PRICE":BITCOIN_PRICEs})
        
        trade_df = pd.concat([trade_df,new_rows],ignore_index=True)
        
        if verbose == 1:
            print(date_time)
            print(f"LONG {[(pair,long_dict[pair]) for pair in long_pairs]}")
            print(f"SHORT {[(pair,short_dict[pair]) for pair in short_pairs]}")
        elif verbose == 0:
            pass
        else:
            raise ValueError("verbose: 0 or 1")
        
        open_unix_time += holding_period * TRADE_TIMEFRAME_IN_MS

    trade_df["cumsum_profit"] = trade_df["profit"].cumsum()
    trade_df["cumsum_long_profit"] = trade_df[trade_df["type"] == "long"]["profit"].cumsum()
    trade_df["cumsum_short_profit"] = trade_df[trade_df["type"] == "short"]["profit"].cumsum()
    trade_df["cumsum_trading_cost"] = trade_df["cost"].cumsum()
    trade_df.fillna(method="ffill", inplace = True)

    if plot == True:
        market_df = ar.make_market_trend(merged_df,f"BTCUSDT_{market_cycle}_MACD")
        plot_results(trade_df,market_df,(trade_start,trade_end))

    return trade_df, merged_df

In [ ]:
trade_df, merged_df = backtest_strategy(
    leverage = 1,
    trade_duration = 900 * 6,
    holding_period = 14 * 6,
    pair_number = 5,
    verbose = 0,
    balance = 100_000,
    trading_fee = 0.0004,
    take_profit_long = 1.5,
    stop_loss_long = 0.5,
    take_profit_short = 0.8,
    stop_loss_short = 0.5,
    altcoin_cycle = "MEDIUM",
    market_cycle = "SLOW",
    plot=True,
)

aggreegate_df = ar.get_aggregate_report(trade_df=trade_df,holding_period=14 * 6,equity = 100_000)
sharpe_ratio = ar.get_sharpe(aggreegate_df)
print(f"Sharpe: {sharpe_ratio}")
ar.analyze_trade(trade_df)